---
format: 
  html:
    toc: false
    page-layout: full
execute:
    echo: false
---

# Building our random forest model

Now that we have loaded and processed our data, we will proceed to build and test our random forest model.


In [12]:
!pip install statsmodels

# load necessary libraries
import os
import requests
import pysal
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import folium
import osmnx as ox
import statsmodels.api as sm
from census import Census
from us import states
from folium import plugins
from sklearn.preprocessing import QuantileTransformer
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [3]:
# load processed data
geo_data = gpd.read_file("geo_data_crashes.geojson")

In [36]:
# Step 1: Prepare the data
# Select columns that start with "crash_", "commute_", "vehicle_", end with "_proportion", and "total_length"
predictor_columns = [col for col in geo_data.columns if col.startswith("crash_") or col.startswith("commute_") or col.startswith("vehicle_") or col.endswith("_proportion") or col == "total_length"]

# Ensure the target variable for 2023 is included
target_column = "crash_2023"  # Replace with the actual target variable if different

# Filter the data to include only the selected columns and the target variable
data = geo_data[predictor_columns + [target_column]]

# Drop rows with missing values
data = data.dropna()

# Step 2: Split the data into training and testing sets
# Use data from 2020 through 2022 for training
train_columns = [col for col in predictor_columns if col.startswith("crash_2020") or col.startswith("crash_2021") or col.startswith("crash_2022") or col.startswith("commute_") or col.startswith("vehicle_") or col.endswith("_proportion") or col == "total_length"]

X_train = data[train_columns]
y_train = data[target_column]

# Use 2023 data for testing
X_test = data[train_columns]
y_test = data[target_column]

y_train = y_train.loc[:, ~y_train.columns.duplicated()]
y_test = y_test.loc[:, ~y_test.columns.duplicated()]

# Step 3: Train a random forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 4: Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Optional: Feature importance
feature_importances = pd.Series(model.feature_importances_, index=train_columns).sort_values(ascending=False)
print("Feature Importances:")
print(feature_importances)

/Users/songj/miniforge3/envs/musa-550-fall-2024/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Mean Squared Error: 0.79088359375
Feature Importances:
crash_2020                  0.288014
crash_2022                  0.256575
crash_2021                  0.113009
residential_proportion      0.043929
commute_public_transport    0.039824
commute_carpool             0.037152
tertiary_proportion         0.034526
commute_drive_alone         0.033792
vehicle_one                 0.031244
total_length                0.030150
commute_walked              0.025357
commute_work_from_home      0.024944
vehicle_none                0.024552
commute_bike                0.016933
dtype: float64


In [20]:
# set up random forest pipeline
forest = make_pipeline(
    StandardScaler(), RandomForestRegressor(n_estimators=100, random_state=42)
)

# generate hyperparameter grid
model_step = "randomforestregressor"
param_grid = {
    f"{model_step}__n_estimators": [5, 10, 15, 20, 30, 50, 100, 200],
    f"{model_step}__max_depth": [2, 5, 7, 9, 13, 21, 33, 51],
}

# create grid and use 3-fold CV
grid = GridSearchCV(forest, param_grid, cv=3, verbose=1)

# run the model
grid.fit(X_train, y_train)

# display optimized hyperparameters
grid.best_params_

# show score
grid.score(X_test, y_test)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


/Users/songj/miniforge3/envs/musa-550-fall-2024/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/songj/miniforge3/envs/musa-550-fall-2024/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/songj/miniforge3/envs/musa-550-fall-2024/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/songj/miniforge3/envs/musa-550-fall-2024/lib/python3.10/site-packages/sklearn/base.py:1

0.7915037209365611

In [37]:
# evaluate the best random forest model
best_random = grid.best_estimator_

# make the predictions
predictions = best_random.predict(X_test)

# absolute error
y_test_array = y_test.values.flatten()

errors = abs(predictions - y_pred)
avg_error = np.mean(errors)

# Add a small epsilon value to y_test_array to avoid division by zero
mape = 100 * np.mean(errors / y_pred)

# accuracy
accuracy = 100 - mape

print("Model Performance")
print(f"Average Absolute Error: {avg_error:0.4f}")
print(f"Accuracy = {accuracy:0.2f}%.")

Model Performance
Average Absolute Error: 0.5511
Accuracy = 57.14%.


In [39]:
# Prepare the data
# Assuming X_train, y_train, X_test, y_test are already defined

# Add a constant to the predictors (intercept term)
X_train_nb = sm.add_constant(X_train)
X_test_nb = sm.add_constant(X_test)

# Fit the Poisson regression model
nb_model = sm.GLM(y_train, X_train_nb, family=sm.families.NegativeBinomial()).fit()

# Make predictions
nb_predictions = nb_model.predict(X_test)

# Calculate the absolute errors
nb_errors = abs(nb_predictions - y_pred)

# Calculate the average error
nb_avg_error = np.mean(nb_errors)

# Mean absolute percentage error
nb_mape = 100 * np.mean(nb_errors / y_pred)

# Accuracy
nb_accuracy = 100 - mape

print("Negative binomial regression model performance")
print(f"Average absolute error: {nb_avg_error:0.4f}")
print(f"Accuracy = {accuracy:0.2f}%")

Poisson Regression Model Performance
Average Absolute Error: 1.4842
Accuracy = 29.24%


/Users/songj/miniforge3/envs/musa-550-fall-2024/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
